In [1]:
%pip install scikit-learn llama-index llama-index-embeddings-ollama pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 🔍 Demonstration: Retrieval-Augmented Generation (RAG) with Cosine Similarity
This notebook showcases how Retrieval-Augmented Generation (RAG) works under the hood by using cosine similarity to retrieve the most relevant text chunks in response to a query.

## 🧾 Test Document
The evaluation is based on a synthetic insurance document generated by GPT-4o (OpenAI). The document includes:

- A fictional insurance contract by Allianz AG
- Customer details (e.g., policyholder: Max Mustermann)
- Key contractual clauses and policy terms
- An unrelated paragraph to simulate noise or misleading context (for testing retrieval robustness)

## 🧠 Embedding Models Tested
To compute similarity and retrieve relevant text chunks, we use vector embeddings from the following models:

- text-embedding-ada-002 (OpenAI)
- Llama3.3-70B (via embedding interface)

## 🧠 Language Synthetization with LLM
To generate an answer, we retrieve the Top-K from the similarity scores, put them with the query in a meaningful context, and pass the context to the LLM:

- GPT-3.5-Turbo-Instruct
- Llama3.3-70B

## ⚙️ Workflow Overview
1. The document is split into manageable text chunks
2. Each chunk is embedded into a vector space
3. A user query is embedded and compared to all chunk vectors using cosine similarity
4. The top-k most relevant chunks are passed to the LLM (Llama3.3:70B or GPT-3.5-Instruct) for response generation

This setup helps illustrate how RAG systems combine information retrieval with language generation to provide grounded and context-aware responses.

In [101]:
from dotenv import load_dotenv
load_dotenv()

True

In [102]:
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

import os

api_key = os.getenv('OPENAI_KEY')

llm = OpenAI(
    temperature=0,
    api_key=api_key,
    model="gpt-3.5-turbo-instruct"
)
embed_model = OpenAIEmbedding(
    model='text-embedding-ada-002',
    api_key=api_key,
)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256

In [73]:
from llama_index.core.readers import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['./data/insurance_contract_example.txt']).load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00, 975.19it/s]


In [103]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=20,
)
chunks = parser.get_nodes_from_documents(documents)
len(chunks)

5

In [104]:
query = "Wer ist der Versicherer und wer ist Versicherungsnehmer? Was wird gehaftet?"
embeddings_document = [
    embed_model.get_text_embedding(chunk.text) for chunk in chunks
]
embedding_query = embed_model.get_text_embedding(query)

In [120]:
import pandas as pd

data=[embedding_query] + embeddings_document
pd.DataFrame(data, columns=[f"dim_{i}" for i in range(len(embedding_query))], index=['query'] + [f'chunk_{i}' for i in range(len(embeddings_document))])

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_1526,dim_1527,dim_1528,dim_1529,dim_1530,dim_1531,dim_1532,dim_1533,dim_1534,dim_1535
query,0.022058,-0.000383,0.020304,-0.029112,-0.030157,0.017517,-0.013511,0.026474,-0.005315,-0.008429,...,-0.022095,-0.014270,0.060364,-0.019470,-0.006973,-0.013399,-0.020776,-0.005225,-0.002555,-0.000626
chunk_0,0.002563,0.012574,0.032025,-0.017141,-0.020726,0.006141,-0.023660,-0.009639,-0.011731,-0.013098,...,0.006562,-0.016371,0.029795,-0.011936,-0.012428,-0.009055,0.006513,0.009832,0.001305,-0.032981
chunk_1,-0.000455,0.002097,-0.000525,-0.051893,-0.027034,0.036227,-0.034518,0.008973,-0.006648,-0.004554,...,-0.005092,-0.036330,0.052670,-0.021311,-0.023888,-0.000178,-0.016391,0.003991,-0.004998,-0.020185
chunk_2,0.004049,-0.003040,0.016405,-0.044796,-0.029327,0.022164,-0.021423,-0.003507,0.013883,-0.012785,...,0.016626,-0.018030,0.038349,-0.009834,-0.029951,-0.008040,0.005518,-0.005814,0.010062,-0.023022
chunk_3,0.011782,0.002652,0.010153,-0.044751,-0.027816,0.008257,-0.017356,0.011201,0.022886,-0.005102,...,-0.009611,-0.010907,0.021597,0.005121,-0.018927,0.000893,0.018123,0.010556,0.008525,0.003372
chunk_4,0.000556,-0.009172,0.006921,-0.041227,-0.022494,0.031773,-0.026531,-0.012689,0.004163,-0.007134,...,0.006238,-0.010363,0.038117,-0.003821,-0.026256,-0.000344,0.020964,0.002751,-0.005025,-0.016037


In [121]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity([embedding_query], embeddings_document)[0]

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([similarities, [chunk.text for chunk in chunks]], index=["Similarity score", "Text"])
df.T

,Similarity score,Text
0,0.799474,"📉 Semantisch irrelevanter Abschnitt (für Demonstration)\nDie durchschnittliche Bearbeitungszeit von Leistungsanträgen im Bereich Zahnzusatzversicherung beträgt laut interner Statistik der Allianz AG etwa 7,3 Werktage, wobei regionale Unterschiede durch standortbezogene Ressourcenverteilung entstehen können.\n\nAllianz Versicherungs-AG\nKöniginstraße 28\n80802 München\nTelefon: 089 3800-0\nE-Mail: service@allianz.de\nWeb: www.allianz.de\n\nVersicherungsschein (Police-Nr."
1,0.846007,"1234 5678 9012)\nVertragsbeginn: 01.07.2025\nVersicherungsnehmer:\nMax Mustermann\nMusterstraße 12\n12345 Musterstadt\nGeburtsdatum: 01.01.1980\nVersichert seit: 01.07.2025\n\n🛡️ Vertragsübersicht\nVersicherungsart:\nPrivathaftpflichtversicherung – Komfortschutz\n\nGeltungsbereich:\nWeltweit (inkl. vorübergehende Auslandsaufenthalte bis 12 Monate)\n\nVersicherungssumme:\n10.000.000 € pauschal für Personen-, Sach- und Vermögensschäden\n\nSelbstbeteiligung:\n150 € je Schadensfall\n\nJährlicher Beitrag:\n120,00 € (inkl."
2,0.848348,"Versicherungssteuer)\n\nZahlweise:\njährlich, per SEPA-Lastschrift\n\nZahlungstermin:\n01.07. jeden Jahres\n\n📋 Wichtige Vertragsbedingungen\nWir, die Versicherer Allianz Versicherungs-AG, haften für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden.\n\nDer Versicherer haftet für den Versicherungsnehmer Max Mustermann, sofern Ansprüche Dritter auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
3,0.854584,"Die Versicherung haftet für folgende Objekte, sofern diese sich im Eigentum oder rechtmäßigen Besitz des Versicherungsnehmers befinden und im Rahmen des Versicherungsvertrages eingeschlossen sind:\n\nWohnräume und Nebengebäude am Wohnort\n\nHaustiere (z. B. Hunde, Katzen – keine gefährlichen Tiere i.S.d. Gesetzes)\n\nBewegliche Gegenstände des täglichen Lebens\n\nSchäden im Rahmen gesetzlicher Haftung im privaten Umfeld\n\nDer Versicherungsschutz beginnt mit dem in der Police genannten Datum, sofern der erste Beitrag rechtzeitig gezahlt wurde.\n\nDie Vertragslaufzeit beträgt 1 Jahr und verlängert sich automatisch, sofern nicht 3 Monate vor Ablauf gekündigt wird.\n\nEs gelten die Allgemeinen Versicherungsbedingungen (AVB PHV 2025)."
4,0.810156,"📌 Kontakt bei Schadenmeldung\nTelefonische Schadenmeldung:\n089 3800-5555 (Mo–Fr, 8–20 Uhr)\n\nOnline:\nwww.allianz.de/schaden-melden\n\nE-Mail:\nschaden@allianz.de\n\n🖊️ Bestätigung\nBitte prüfen Sie die Angaben zu Ihrer Versicherung sorgfältig. Bei Fragen oder Änderungswünschen wenden Sie sich gerne an unseren Kundenservice.\n\nMit freundlichen Grüßen\nIhre Allianz Versicherungs-AG\n\nUnterschrift maschinell erstellt – gültig ohne Unterschrift\n\n🔒 Datenschutz-Hinweis\nIhre personenbezogenen Daten werden gemäß den geltenden Datenschutzvorschriften, insbesondere der DSGVO, verarbeitet. Weitere Informationen finden Sie unter www.allianz.de/datenschutz.\n\nHinweis: Dieses Dokument ist ein fiktives Beispiel und dient ausschließlich Demonstrationszwecken. Es besteht kein tatsächlicher Versicherungsvertrag."


In [122]:
sorted = df.T.sort_values(by=["Similarity score"], ascending=False)
sorted

,Similarity score,Text
3,0.854584,"Die Versicherung haftet für folgende Objekte, sofern diese sich im Eigentum oder rechtmäßigen Besitz des Versicherungsnehmers befinden und im Rahmen des Versicherungsvertrages eingeschlossen sind:\n\nWohnräume und Nebengebäude am Wohnort\n\nHaustiere (z. B. Hunde, Katzen – keine gefährlichen Tiere i.S.d. Gesetzes)\n\nBewegliche Gegenstände des täglichen Lebens\n\nSchäden im Rahmen gesetzlicher Haftung im privaten Umfeld\n\nDer Versicherungsschutz beginnt mit dem in der Police genannten Datum, sofern der erste Beitrag rechtzeitig gezahlt wurde.\n\nDie Vertragslaufzeit beträgt 1 Jahr und verlängert sich automatisch, sofern nicht 3 Monate vor Ablauf gekündigt wird.\n\nEs gelten die Allgemeinen Versicherungsbedingungen (AVB PHV 2025)."
2,0.848348,"Versicherungssteuer)\n\nZahlweise:\njährlich, per SEPA-Lastschrift\n\nZahlungstermin:\n01.07. jeden Jahres\n\n📋 Wichtige Vertragsbedingungen\nWir, die Versicherer Allianz Versicherungs-AG, haften für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden.\n\nDer Versicherer haftet für den Versicherungsnehmer Max Mustermann, sofern Ansprüche Dritter auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
1,0.846007,"1234 5678 9012)\nVertragsbeginn: 01.07.2025\nVersicherungsnehmer:\nMax Mustermann\nMusterstraße 12\n12345 Musterstadt\nGeburtsdatum: 01.01.1980\nVersichert seit: 01.07.2025\n\n🛡️ Vertragsübersicht\nVersicherungsart:\nPrivathaftpflichtversicherung – Komfortschutz\n\nGeltungsbereich:\nWeltweit (inkl. vorübergehende Auslandsaufenthalte bis 12 Monate)\n\nVersicherungssumme:\n10.000.000 € pauschal für Personen-, Sach- und Vermögensschäden\n\nSelbstbeteiligung:\n150 € je Schadensfall\n\nJährlicher Beitrag:\n120,00 € (inkl."
4,0.810156,"📌 Kontakt bei Schadenmeldung\nTelefonische Schadenmeldung:\n089 3800-5555 (Mo–Fr, 8–20 Uhr)\n\nOnline:\nwww.allianz.de/schaden-melden\n\nE-Mail:\nschaden@allianz.de\n\n🖊️ Bestätigung\nBitte prüfen Sie die Angaben zu Ihrer Versicherung sorgfältig. Bei Fragen oder Änderungswünschen wenden Sie sich gerne an unseren Kundenservice.\n\nMit freundlichen Grüßen\nIhre Allianz Versicherungs-AG\n\nUnterschrift maschinell erstellt – gültig ohne Unterschrift\n\n🔒 Datenschutz-Hinweis\nIhre personenbezogenen Daten werden gemäß den geltenden Datenschutzvorschriften, insbesondere der DSGVO, verarbeitet. Weitere Informationen finden Sie unter www.allianz.de/datenschutz.\n\nHinweis: Dieses Dokument ist ein fiktives Beispiel und dient ausschließlich Demonstrationszwecken. Es besteht kein tatsächlicher Versicherungsvertrag."
0,0.799474,"📉 Semantisch irrelevanter Abschnitt (für Demonstration)\nDie durchschnittliche Bearbeitungszeit von Leistungsanträgen im Bereich Zahnzusatzversicherung beträgt laut interner Statistik der Allianz AG etwa 7,3 Werktage, wobei regionale Unterschiede durch standortbezogene Ressourcenverteilung entstehen können.\n\nAllianz Versicherungs-AG\nKöniginstraße 28\n80802 München\nTelefon: 089 3800-0\nE-Mail: service@allianz.de\nWeb: www.allianz.de\n\nVersicherungsschein (Police-Nr."


In [123]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=chunks, embed_model=embed_model)

In [124]:
top_k = 1 # default is 2, can be extended or reduced!

query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k, llm=llm)
response = query_engine.query(query)

In [125]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"\nDer Versicherer ist nicht explizit genannt, da es sich um ein Beispiel handelt. Der Versicherungsnehmer ist die Person, die den Vertrag abschließt und somit die Versicherung in Anspruch nehmen kann. Es wird für Wohnräume, Nebengebäude, Haustiere, bewegliche Gegenstände und Schäden im privaten Umfeld gehaftet."


In [126]:
top_k = 2 # default is 2, can be extended or reduced!

query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k)
response = query_engine.query(query)

In [127]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"\nDer Versicherer ist die Allianz Versicherungs-AG und der Versicherungsnehmer ist Max Mustermann. Der Versicherer haftet für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden."


### 📄 Querying and Embedding with Llama3.3-70B

Here, we demonstrate the use of the Llama3.3-70B model for Retrieval-Augmented Generation (RAG). The workflow involves embedding both the documents and the user query into a vector space using the `OllamaEmbedding` model. Cosine similarity is then computed to retrieve the most relevant text chunks. The top-k chunks are passed to the Llama3.3-70B model, which synthesizes a context-aware response.

For the query, *"Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"*, the system retrieves relevant chunks from the embedded insurance contract and generates a response using Llama3.3-70B:

In [88]:
%pip install llama-index-llms-ollama

Note: you may need to restart the kernel to use updated packages.


In [131]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.settings import Settings
from dotenv import load_dotenv

load_dotenv()
import os

base_url = os.getenv('OLLAMA_URL', 'http://localhost:11434')

llm = Ollama(
    model='llama3.3:70b',
    base_url=base_url,
    temperature=0,
    request_timeout=42069,
)
embed_model = OllamaEmbedding(
    model_name='llama3.3:70b',
    base_url=base_url
)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256

In [132]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=['./data/insurance_contract_example.txt']).load_data(show_progress=True)

parser = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=20,
)
chunks = parser.get_nodes_from_documents(documents)
len(chunks)

Loading files: 100%|██████████| 1/1 [00:00<00:00, 90.93it/s]


5

In [133]:
query = "Wer ist der Versicherer und wer ist Versicherungsnehmer? Was wird gehaftet?"
embeddings_document = [
    embed_model.get_text_embedding(chunk.text) for chunk in chunks
]
embedding_query = embed_model.get_text_embedding(query)

In [ ]:
import pandas as pd

data = [embedding_query] + embeddings_document


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity([embedding_query], embeddings_document)[0]
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([similarities, [chunk.text for chunk in chunks]], index=["Similarity score", "Text"])
df.T.sort_values(by=["Similarity score"], ascending=False)

In [134]:
import pandas as pd

data=[embedding_query] + embeddings_document
pd.DataFrame(data, columns=[f"dim_{i}" for i in range(len(embedding_query))], index=['query'] + [f'chunk_{i}' for i in range(len(embeddings_document))])

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_8182,dim_8183,dim_8184,dim_8185,dim_8186,dim_8187,dim_8188,dim_8189,dim_8190,dim_8191
query,1.809957,1.160993,-0.178680,-0.291632,0.031140,-3.526653,0.165268,0.855529,2.118510,-1.071882,...,-1.895919,0.633318,2.164165,-1.825784,-2.202570,-0.078867,-2.166760,-0.302955,-0.755186,0.641632
chunk_0,3.403073,0.955625,0.282160,0.559295,0.154261,-2.681694,2.625498,-0.944338,-1.938615,0.866476,...,-0.263736,0.206389,0.957049,-4.213208,-6.976963,2.613168,-3.147981,-0.901458,-0.114647,-0.915522
chunk_1,-0.908719,1.859349,1.179339,1.347932,-1.418475,-2.734751,1.348252,2.565235,1.381842,1.679480,...,0.903621,-0.642808,0.419378,-0.199260,0.006197,1.573569,-3.167247,2.303285,3.952700,1.461883
chunk_2,2.760553,2.744679,-2.149384,-0.606698,0.903329,-2.224410,0.380152,1.516252,1.619553,0.456659,...,-2.067571,1.001213,0.862720,-0.518825,-1.814864,1.537197,-1.882667,2.411174,-1.630921,2.108653
chunk_3,3.675587,2.137812,-0.822753,-0.002808,0.579675,-1.724955,0.820626,1.376261,0.482148,-0.693487,...,-2.062442,0.064052,1.663474,-0.279410,-2.134921,1.541156,-2.272158,2.156033,-1.815461,2.330843
chunk_4,3.830493,3.235324,-0.978477,-1.270671,2.069952,-1.335393,-0.702189,1.032326,-0.342774,0.062674,...,-0.612683,0.275802,1.155553,0.921935,-0.999526,2.018221,-1.484425,0.591483,1.542960,1.489634


In [135]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=chunks, embed_model=embed_model)

top_k = 1 # default is 2, can be extended or reduced!
query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k, llm=llm)
response = query_engine.query(query)

In [136]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"Der Versicherer ist die Allianz Versicherungs-AG und der Versicherungsnehmer ist Max Mustermann. Der Versicherer haftet für den Versicherungsnehmer gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden, sowie für Ansprüche Dritter auf gesetzlicher Grundlage im Rahmen der Privathaftpflichtversicherung."


In [137]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=chunks, embed_model=embed_model)

top_k = 2 # default is 2, can be extended or reduced!
query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k, llm=llm)
response = query_engine.query(query)

In [138]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"Der Versicherer ist die Allianz Versicherungs-AG, während der Versicherungsnehmer Max Mustermann ist. Der Versicherer haftet für den Versicherungsnehmer gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden, sofern Ansprüche auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
